In [1]:
import json
import os
from collections import OrderedDict
import pickle

In [2]:
# AI 허브 전체 데이터 폴더 경로 
# 라벨 : json , 원천 : image
# compath : 데이터 공통 경로

com_path = 'D:/Project/Inha/Data/반려동물 구분을 위한 동물 영상/Training/DOG)ETC/'
path =  com_path+ '[라벨]'
img_path =  com_path + '[원천]'

# 공통경로에 있는 폴더 이름들 출력
file_list = os.listdir(com_path)
print(len(file_list))

10


In [3]:
# os.listdir로 폴더명들 불러오면 윈도우 파일 탐색기에서 보이는 순서랑 달라져서 같게 하기 위해서 정렬
sorted(file_list)

['[라벨]BODYSCRATCH',
 '[라벨]MOUNTING',
 '[라벨]SIT',
 '[라벨]TURN',
 '[라벨]WALKRUN',
 '[원천]BODYSCRATCH',
 '[원천]MOUNTING',
 '[원천]SIT',
 '[원천]TURN',
 '[원천]WALKRUN']

In [4]:
# 라벨링 파일(json) 과 이미지 경로 추출

def labeling(label_path, img_path ,target ):

    data_list = []
    
    
    # 라벨링 파일 오픈
    with open (label_path + target, "r" , encoding="utf8") as f:
        data = json.load(f)

        # 현재 파일명 추출
        video_name = target[:-5] + '/'
        
        # 이미지 경로에서 현재 파일명과 같은 이미지 폴더 경로 추출
        img_folder = os.listdir(img_path + video_name)

        # 저장할 데이터 만들기 
        for anno in data['annotations']:
            for img in img_folder :
                # 라벨링 파일에 있는 img 이름에 맞게 이미지 파일 경로 추출
                if(anno['frame_number'] == int(img.split('_')[1])):
                    row = []
                    row.append(video_name+img)
                    # 라벨링 파일에서 키포인트만 추출
                    for i in anno['keypoints']:
                        try:
                            row.append((anno['keypoints'][i]['x']))
                            row.append((anno['keypoints'][i]['y']))
                        except:
                            row.append(None)
                            row.append(None)

                    data_list.append(row)
    return data_list
                                   
                
            

In [5]:
files_list = os.listdir(com_path)
data = []

for files in range(int(len(files_list)/2)) :    
    
    #동작 카테고리 추출 해서 출력
    category = files_list[files][4:]
    print(category)
    
    # 라벨링 경로와 이미지 경로 조합
    path = com_path + '[라벨]' + category + '/' + category +'/' 
    img_path = com_path + '[원천]' + category + '/' + category +'/' 
    
    file_list = os.listdir(path)

    # 경로를 labeling에 전달해서 데이터 초기값 추출
    data_list = labeling(path,img_path,file_list[0])

    
    # 경로를 labeling에 전달해서 데이터 나머지 추출
    for file in file_list[1:] :
        try :
            data_list = data_list + labeling(path,img_path,file)
        except:
            print(file)

    data = data + data_list

BODYSCRATCH
MOUNTING
SIT
TURN
WALKRUN


In [6]:
import pandas as pd

# 오류난 데이터들 제외
del_list = [1220,1169,1295,1342,1168,1208,1233,1094,1201,1174,1148,1153]

# 예시 코드에 들어갈 수 있게 dataframe 생성
df = pd.DataFrame(data, columns=['image','0_x','0_y','1_x','1_y','2_x','2_y','3_x','3_y','4_x','4_y','5_x','5_y','6_x','6_y','7_x','7_y','8_x','8_y','9_x','9_y','10_x','10_y','11_x','11_y','12_x','12_y','13_x','13_y','14_x','14_y'])

# df = df.fillna(0)
# null 값은 제외하고 저장
df.dropna(inplace = True)

# 오류 났었던 데이터를 제외하기 위한 코드 
x_cols = ['0_x', '1_x', '2_x', '3_x', '4_x', '5_x', '6_x', '7_x', '8_x', '9_x', '10_x', '11_x', '12_x', '13_x', '14_x']
y_cols = ['0_y', '1_y', '2_y', '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y', '12_y', '13_y', '14_y']
clean_df = df.drop(df[(df[x_cols].min(axis=1) >= df[x_cols].max(axis=1)) | (df[y_cols].min(axis=1) >= df[y_cols].max(axis=1)) | df[y_cols].max(axis=1).isin(del_list)].index)


In [7]:
df

,image,0_x,0_y,1_x,1_y,2_x,2_y,3_x,3_y,4_x,...,10_x,10_y,11_x,11_y,12_x,12_y,13_x,13_y,14_x,14_y
416,20201202_dog-bodyscratch-009198.mp4/frame_149_...,364.0,761.0,380.0,676.0,396.0,795.0,364.0,795.0,544.0,...,583.0,800.0,517.0,1006.0,475.0,905.0,890.0,964.0,820.0,1027.0
417,20201202_dog-bodyscratch-009198.mp4/frame_155_...,356.0,758.0,374.0,672.0,414.0,772.0,356.0,792.0,544.0,...,525.0,832.0,526.0,998.0,467.0,916.0,890.0,964.0,818.0,1019.0
418,20201202_dog-bodyscratch-009198.mp4/frame_161_...,346.0,774.0,367.0,676.0,411.0,781.0,356.0,809.0,536.0,...,523.0,835.0,517.0,994.0,457.0,920.0,890.0,964.0,818.0,1019.0
504,dog-bodyscratch-002900/frame_0_timestamp_0.jpg,373.0,903.0,424.0,818.0,501.0,945.0,385.0,993.0,648.0,...,799.0,1139.0,808.0,1193.0,809.0,1208.0,709.0,848.0,940.0,1061.0
505,dog-bodyscratch-002900/frame_12_timestamp_400.jpg,352.0,899.0,418.0,811.0,485.0,945.0,373.0,995.0,646.0,...,806.0,1128.0,800.0,1193.0,805.0,1210.0,684.0,847.0,934.0,1041.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226113,dog-walkrun-102913/frame_130_timestamp_5200.jpg,255.0,1365.0,222.0,1236.0,320.0,1272.0,277.0,1375.0,423.0,...,862.0,960.0,960.0,1001.0,949.0,1023.0,910.0,650.0,966.0,416.0
226114,dog-walkrun-102913/frame_135_timestamp_5400.jpg,461.0,1321.0,482.0,1174.0,412.0,1388.0,464.0,1326.0,582.0,...,833.0,975.0,794.0,1056.0,796.0,1057.0,898.0,699.0,1049.0,527.0
226115,dog-walkrun-102913/frame_140_timestamp_5600.jpg,726.0,1204.0,715.0,1011.0,636.0,1152.0,696.0,1220.0,695.0,...,784.0,1102.0,787.0,1079.0,792.0,1063.0,814.0,699.0,891.0,545.0
226116,dog-walkrun-102913/frame_145_timestamp_5800.jpg,684.0,1194.0,674.0,1006.0,609.0,1161.0,647.0,1200.0,619.0,...,852.0,627.0,674.0,665.0,673.0,672.0,667.0,672.0,637.0,521.0


In [8]:
clean_df

,image,0_x,0_y,1_x,1_y,2_x,2_y,3_x,3_y,4_x,...,10_x,10_y,11_x,11_y,12_x,12_y,13_x,13_y,14_x,14_y
416,20201202_dog-bodyscratch-009198.mp4/frame_149_...,364.0,761.0,380.0,676.0,396.0,795.0,364.0,795.0,544.0,...,583.0,800.0,517.0,1006.0,475.0,905.0,890.0,964.0,820.0,1027.0
417,20201202_dog-bodyscratch-009198.mp4/frame_155_...,356.0,758.0,374.0,672.0,414.0,772.0,356.0,792.0,544.0,...,525.0,832.0,526.0,998.0,467.0,916.0,890.0,964.0,818.0,1019.0
418,20201202_dog-bodyscratch-009198.mp4/frame_161_...,346.0,774.0,367.0,676.0,411.0,781.0,356.0,809.0,536.0,...,523.0,835.0,517.0,994.0,457.0,920.0,890.0,964.0,818.0,1019.0
504,dog-bodyscratch-002900/frame_0_timestamp_0.jpg,373.0,903.0,424.0,818.0,501.0,945.0,385.0,993.0,648.0,...,799.0,1139.0,808.0,1193.0,809.0,1208.0,709.0,848.0,940.0,1061.0
505,dog-bodyscratch-002900/frame_12_timestamp_400.jpg,352.0,899.0,418.0,811.0,485.0,945.0,373.0,995.0,646.0,...,806.0,1128.0,800.0,1193.0,805.0,1210.0,684.0,847.0,934.0,1041.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226113,dog-walkrun-102913/frame_130_timestamp_5200.jpg,255.0,1365.0,222.0,1236.0,320.0,1272.0,277.0,1375.0,423.0,...,862.0,960.0,960.0,1001.0,949.0,1023.0,910.0,650.0,966.0,416.0
226114,dog-walkrun-102913/frame_135_timestamp_5400.jpg,461.0,1321.0,482.0,1174.0,412.0,1388.0,464.0,1326.0,582.0,...,833.0,975.0,794.0,1056.0,796.0,1057.0,898.0,699.0,1049.0,527.0
226115,dog-walkrun-102913/frame_140_timestamp_5600.jpg,726.0,1204.0,715.0,1011.0,636.0,1152.0,696.0,1220.0,695.0,...,784.0,1102.0,787.0,1079.0,792.0,1063.0,814.0,699.0,891.0,545.0
226116,dog-walkrun-102913/frame_145_timestamp_5800.jpg,684.0,1194.0,674.0,1006.0,609.0,1161.0,647.0,1200.0,619.0,...,852.0,627.0,674.0,665.0,673.0,672.0,667.0,672.0,637.0,521.0


In [9]:
clean_df[50000:100000]

,image,0_x,0_y,1_x,1_y,2_x,2_y,3_x,3_y,4_x,...,10_x,10_y,11_x,11_y,12_x,12_y,13_x,13_y,14_x,14_y
59694,dog-bodylower-069348/frame_126_timestamp_4200.jpg,928.0,1043.0,900.0,971.0,887.0,1068.0,920.0,1064.0,646.0,...,245.0,912.0,210.0,985.0,143.0,929.0,125.0,726.0,110.0,923.0
59695,dog-bodylower-069348/frame_132_timestamp_4400.jpg,936.0,1039.0,891.0,964.0,892.0,1066.0,930.0,1059.0,654.0,...,254.0,902.0,221.0,985.0,182.0,954.0,104.0,740.0,329.0,912.0
59696,dog-bodylower-069348/frame_138_timestamp_4600.jpg,939.0,1040.0,896.0,963.0,896.0,1066.0,932.0,1060.0,655.0,...,285.0,908.0,214.0,984.0,183.0,957.0,115.0,739.0,120.0,920.0
59697,dog-bodylower-069348/frame_144_timestamp_4800.jpg,943.0,1037.0,899.0,964.0,898.0,1068.0,935.0,1060.0,658.0,...,345.0,932.0,210.0,983.0,219.0,983.0,115.0,744.0,124.0,911.0
59698,dog-bodylower-069348/frame_150_timestamp_5000.jpg,941.0,1038.0,899.0,965.0,898.0,1069.0,934.0,1061.0,658.0,...,348.0,935.0,210.0,982.0,223.0,982.0,138.0,726.0,125.0,916.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112242,dog-bodyshake-031449/frame_0_timestamp_0.jpg,258.0,831.0,274.0,669.0,386.0,843.0,290.0,876.0,638.0,...,898.0,818.0,953.0,756.0,946.0,758.0,756.0,617.0,839.0,388.0
112243,dog-bodyshake-031449/frame_12_timestamp_400.jpg,290.0,853.0,276.0,684.0,378.0,876.0,326.0,901.0,598.0,...,879.0,831.0,940.0,776.0,927.0,773.0,745.0,624.0,827.0,395.0
112244,dog-bodyshake-031449/frame_102_timestamp_3400.jpg,345.0,870.0,355.0,703.0,378.0,921.0,346.0,923.0,594.0,...,807.0,882.0,864.0,830.0,853.0,826.0,656.0,661.0,810.0,542.0
112245,dog-bodyshake-031449/frame_108_timestamp_3600.jpg,281.0,828.0,313.0,660.0,344.0,884.0,296.0,885.0,654.0,...,788.0,871.0,846.0,824.0,838.0,822.0,654.0,645.0,749.0,450.0


In [9]:
# 제외한 데이터 csv 파일로 저장
clean_df.to_csv("filename.csv", mode='w')

## 데이터 확인용 테스트 코드

In [19]:


import numpy as np

keypoints =df.iloc[0, 1:].values.reshape(-1, 2).astype(np.int64)

x = keypoints[: ,0]
y = keypoints[: ,1]

In [ ]:
x1, y1 = min(keypoints[: ,1][keypoints[: ,1] != 0.]), min(y[y != 0.])
x2, y2 = max(keypoints[:, 0]), max(keypoints[:, 1])

print(x1,y1,x2,y2)

In [ ]:
import pickle

# 데이터 로드
with open('train_class_changed.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data

In [ ]:
# df = df.fillna(0)

df = pd.DataFrame(data, columns=['image','0_x','0_y','1_x','1_y','2_x','2_y','3_x','3_y','4_x','4_y','5_x','5_y','6_x','6_y','7_x','7_y','8_x','8_y','9_x','9_y','10_x','10_y','11_x','11_y','12_x','12_y','13_x','13_y','14_x','14_y'])
x_cols = ['0_x', '1_x', '2_x', '3_x', '4_x', '5_x', '6_x', '7_x', '8_x', '9_x', '10_x', '11_x', '12_x', '13_x', '14_x']
y_cols = ['0_y', '1_y', '2_y', '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y', '12_y', '13_y', '14_y']
df.dropna(inplace = True)
df[df[x_cols].min(axis=1)]
